In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier

In [15]:
import warnings
warnings.filterwarnings("ignore")

# Data

In [16]:
train = pd.read_csv("location_train.csv")
test = pd.read_csv("location_test.csv")

In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Columns: 448 entries, ID to 446
dtypes: int64(448)
memory usage: 13.7 MB


In [18]:
train.head()

,ID,class,1,2,3,4,5,6,7,8,...,437,438,439,440,441,442,443,444,445,446
0,0,11,0,0,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,8,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,4,3,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [19]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Columns: 447 entries, ID to 446
dtypes: int64(447)
memory usage: 3.4 MB


In [20]:
test.head()

,ID,1,2,3,4,5,6,7,8,9,...,437,438,439,440,441,442,443,444,445,446
0,4000,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,4001,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,4002,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4003,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,4004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [21]:
X_train = train.drop(["ID", "class"], axis=1)
y_train = train["class"]

X_test = test.drop(["ID"], axis=1)

# Model selection

In [22]:
# Grid Search

cv = 10         # number of folds
verbose = 1     # information shown during training

## Decision Tree

In [23]:
parameters = {
    "criterion":["gini", "entropy"],
    "splitter":["best", "random"], 
    "max_depth":["None", 5, 10, 20, 50, 100],
    "max_features":["auto", "sqrt", "log2"]}    
model = GridSearchCV(DecisionTreeClassifier(), parameters, cv=cv, verbose=verbose, scoring="f1_weighted")
model.fit(X_train, y_train)

results = pd.DataFrame(model.cv_results_)
results = results[["param_criterion", "param_splitter", "param_max_depth", "param_max_features", "mean_test_score"]]
results.sort_values(["mean_test_score"], ascending=False).head(10)

Fitting 10 folds for each of 72 candidates, totalling 720 fits


,param_criterion,param_splitter,param_max_depth,param_max_features,mean_test_score
27,gini,random,50,sqrt,0.222559
18,gini,best,20,auto,0.221337
19,gini,random,20,auto,0.220329
25,gini,random,50,auto,0.217374
33,gini,random,100,sqrt,0.215563
24,gini,best,50,auto,0.214986
20,gini,best,20,sqrt,0.212889
21,gini,random,20,sqrt,0.212489
32,gini,best,100,sqrt,0.210919
26,gini,best,50,sqrt,0.208369


# Final model

In [24]:
best_model = model.best_estimator_
best_model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=50, max_features='sqrt', splitter='random')

In [25]:
predictions = pd.DataFrame(test["ID"])
predictions["class"] = best_model.predict(X_test)

In [26]:
predictions.to_csv("submission.csv", index=False)